## Maximum Likelyhood for PGM

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Class definitions - RV, Distribution and Category

In [19]:
class RV:
    def __init__(self, cattup,name):
        self.name = name
        self.categories = cattup
        self.catcountdict = {}
        for category in self.categories:
            self.catcountdict[category] = Category(category)
    def __str__(self):
        return_string = " \t < RV: " + self.name + " > \n"
        count = 0
        for category in self.catcountdict:
            return_string = return_string + str(category) + "\t\t"+ str(self.catcountdict[category].count) + "\n"
            count = count + self.catcountdict[category].count
        return_string = return_string + "total count: " + str(count)
        return return_string
    def clear_count(self):
        for cat in self.catcountdict:
            self.catcountdict[cat].count = 0
        
class Distribution:
    def __init__(self,randomvar):
        self.name = randomvar.name
        categories = randomvar.categories
        catcountdict = randomvar.catcountdict
        distdict = {}
        total = 0
        for category in categories:
            total = total + catcountdict[category].count
        for category in categories:
            distdict[category] = catcountdict[category].count/(1.0*total)
        
        self.categories = categories
        self.distdict = distdict
        self.total = total
    def __str__(self):
        return_string = " \t < Distribution: " + self.name + " > \n"
        for category in self.distdict:
            return_string = return_string + str(category) + "\t\t"+ str(self.distdict[category]) + "\n"
        return return_string
    def to_csv(self):
        table = []
        for category in self.distdict:
            table.append([category,self.distdict[category]])
        df = pd.DataFrame(data=table, columns=['category','prob'])
        name = "prior_" + self.name + ".csv"
        df.to_csv(name,sep=',')
    def to_emdw_csv(self):
        table = []
        ind = 0
        for category in self.distdict:
            table.append([ind,self.distdict[category]])
            ind = ind +1
        df = pd.DataFrame(data=table)
        name = "prior_" + self.name + ".csv"
        print(df.to_string(index=False))
        df.to_csv(name,sep=',',index=False,header=False)

class Category:
    def __init__(self,category,catcount=0):
        self.name = category
        self.count = catcount
    def increment(self):
        self.count = self.count + 1
        




### Read in the graph

In [20]:
x = pd.read_csv('car2.csv')
# x = pd.read_csv('g.csv')


### Store the possible categories per RV

In [4]:
# find out how many categories each RV can take
# rvcategorydict = {}
# for rv in x.columns:
#     rvcategorydict[rv] = tuple(set([item[0] for item in x[[rv]].values]))

rvcategorydict = {'safety': ('low', 'med', 'high'), 'lug_boot': ('small', 'med', 'big'), 'persons': ('2', '4','more'), 'verdict': ('unacc', 'acc', 'good', 'vgood'), 'doors': ('2', '4', '5more', '3'), 'buying': ('vhigh', 'high', 'med', 'low'), 'maint': ('vhigh', 'high', 'med', 'low')}

for rv in rvcategorydict:
    print(rv,rvcategorydict[rv])
    
emdwrvcategorydict = {}
for rv, categories in rvcategorydict.items():
    emdwcategorydict = {}
    for ind, category in enumerate(categories):
        emdwcategorydict[category] = ind
    emdwrvcategorydict[rv] = emdwcategorydict
    
# for rv, emdwcategorydict in emdwrvcategorydict.items():
#     for category,ind in emdwcategorydict.items():
#         print(category,ind)

# for child,parents in family.items():
#     print("------->",child)
#     for rv in parents:
#         print("--------------------->",rv)
#         for category in rvcategorydict[rv]:
#             print(category,emdwrvcategorydict[rv][category])
        
        


maint ('vhigh', 'high', 'med', 'low')
doors ('2', '4', '5more', '3')
buying ('vhigh', 'high', 'med', 'low')
lug_boot ('small', 'med', 'big')
safety ('low', 'med', 'high')
persons ('2', '4', 'more')
verdict ('unacc', 'acc', 'good', 'vgood')


### Set up the RV dictionary

In [5]:
rvdictionary = {}

for rv in rvcategorydict:
    rvdictionary[rv] = RV(rvcategorydict[rv],name=rv)
    

## Calculate the prior probabilities

In [6]:
for rv in x.columns:
    for item in x[[rv]].values:
        rvdictionary[rv].catcountdict[item[0]].increment()

distributiondict = {}

for rv in rvdictionary:
    distributiondict[rv] = Distribution(rvdictionary[rv])

# for rv in rvdictionary:
#     print(rvdictionary[rv])
    
for rv in distributiondict:
    print(distributiondict[rv])
    Distribution.to_emdw_csv(distributiondict[rv])

 	 < Distribution: maint > 
med		0.22676200204290092
vhigh		0.27681307456588355
low		0.22063329928498468
high		0.27579162410623087

0         1
0  0.226762
1  0.276813
2  0.220633
3  0.275792
 	 < Distribution: lug_boot > 
small		0.33401430030643514
med		0.33350357507660877
big		0.3324821246169561

0         1
0  0.334014
1  0.333504
2  0.332482
 	 < Distribution: doors > 
3		0.24974463738508682
2		0.278855975485189
4		0.23442288049029622
5more		0.236976506639428

0         1
0  0.249745
1  0.278856
2  0.234423
3  0.236977
 	 < Distribution: buying > 
med		0.22063329928498468
vhigh		0.27425944841675176
low		0.22063329928498468
high		0.2844739530132789

0         1
0  0.220633
1  0.274259
2  0.220633
3  0.284474
 	 < Distribution: safety > 
med		0.33299284984678246
low		0.33401430030643514
high		0.33299284984678246

0         1
0  0.332993
1  0.334014
2  0.332993
 	 < Distribution: persons > 
2		0.33963227783452504
4		0.33146067415730335
more		0.3289070480081716

0         1
0  0.339632

## Maximum likelyhood

### Define the graph structure

In [13]:
# family = {'C':['A','B'], 'D':['C'],'F':['D','B','C']}
# family = {'verdict':['buying','maint', 'doors', 'persons', 'lug_boot', 'safety']}
family = {'verdict':['buying', 'doors', 'persons', 'safety'],'buying':['maint','lug_boot']}

In [14]:
for child in family:
    print("child: ",child,"\nparents: ", family[child])

child:  buying 
parents:  ['maint', 'lug_boot']
child:  verdict 
parents:  ['buying', 'doors', 'persons', 'safety']


### Compute the maximum likelyhood counts

In [15]:
class ChildContainer:
    def __init__(self,rvobj, childcategorydict,childrvdict):
        self.rvobj = rvobj
        self.catdict = childcategorydict

In [16]:
# dictionaries that will contain the data to calculate MLE for the specific graph structure   
childrvdict = {}
childtotals = {}
childsettotals = {}
graphdict = {}
graphsetdict = {}
# print(childrvdict,childcategorydict)

for rv, item in rvdictionary.items():
    item.clear_count()

for child, parents in family.items():
    childcategorydict = {}
    childsetcategorydict = {}
    dtlist = []
    partuplist = []
    print("child: ",child)
    
    for item in x[[*parents]].values:
        partuplist.append((tuple(parents),tuple(item)))
    childtotals[child] = partuplist
    childsettotals[child] = set(partuplist)
    print(len(set(partuplist)),len(partuplist))
   
    for category in rvcategorydict[child]:
        partuplistcat = []
        print("category: ",category)
        childcategorydict[category] = {}

        for (i,item) in enumerate(x[[*parents]].values):
            if x[[child]].values[i][0] == category:
                partuplistcat.append((tuple(parents),tuple(item)))
        childcategorydict[category] = partuplistcat
        childsetcategorydict[category] = set(partuplistcat)
        print(len(set(childcategorydict[category])),len(childcategorydict[category]))
    
    graphdict[child] = childcategorydict
    graphsetdict[child] = childcategorydict
    
# dictionaries to hold the prelim counts
countdicttotals = {}
countcategorydict = {}

# initializing counts to 0
for child, partuplist in childsettotals.items():
    for partup in partuplist:
        countdicttotals[partup] = (0, child)
        
# sum all occurances of each parent combination for each child   
for child, partuplist in childtotals.items():
    for partup in partuplist:
        countdicttotals[partup] = (countdicttotals[partup][0] + 1,child)
        if countdicttotals[partup][0] > 3:
            print(countdicttotals[partup])
        

        
for child, childcategorydict in graphdict.items():
    for category, partuplist in childcategorydict.items():
        for partup in partuplist:
            countcategorydict[partup] = (0, child, category)


for child, childcategorydict in graphdict.items():
    for category, partuplist in childcategorydict.items():
        for partup in partuplist:
            if category == countcategorydict[partup][2]:
                countcategorydict[partup] = (countcategorydict[partup][0] + 1, child,category)

mledict = {}      
for child, childcategorydict in graphdict.items():
    childmledict = {}
    for category, partuplist in childcategorydict.items():
        for partup in partuplist:
            newtup = (tuple([child]+[par for par in partup[0]]),tuple([category]+[cat for cat in partup[1]]))
            childmledict[newtup] = countcategorydict[partup][0]/countdicttotals[partup][0]
            
    mledict[child] = childmledict

# for entry,condprob in mledict.items():
#     print(entry,condprob)

child:  buying
12 1958
category:  vhigh
12 537
category:  high
12 557
category:  med
12 432
category:  low
12 432
child:  verdict
144 1958
category:  unacc
130 1401
category:  acc
72 423
category:  good
32 69
category:  vgood
16 65
(4, 'buying')
(5, 'buying')
(6, 'buying')
(4, 'buying')
(5, 'buying')
(6, 'buying')
(4, 'buying')
(5, 'buying')
(6, 'buying')
(7, 'buying')
(8, 'buying')
(9, 'buying')
(7, 'buying')
(8, 'buying')
(9, 'buying')
(7, 'buying')
(8, 'buying')
(9, 'buying')
(10, 'buying')
(11, 'buying')
(12, 'buying')
(10, 'buying')
(11, 'buying')
(12, 'buying')
(10, 'buying')
(13, 'buying')
(14, 'buying')
(15, 'buying')
(13, 'buying')
(14, 'buying')
(15, 'buying')
(11, 'buying')
(12, 'buying')
(13, 'buying')
(16, 'buying')
(17, 'buying')
(18, 'buying')
(16, 'buying')
(17, 'buying')
(18, 'buying')
(14, 'buying')
(15, 'buying')
(16, 'buying')
(19, 'buying')
(20, 'buying')
(21, 'buying')
(19, 'buying')
(20, 'buying')
(21, 'buying')
(17, 'buying')
(18, 'buying')
(19, 'buying')
(22, '

### Print the MLE conditional probabilities for each child

In [17]:
familytable = {}
familycolumns = {}
emdwfamilytable = {}

for child, parents in family.items():
    familycolumns[child] = [child] + parents

for child, childmledict in mledict.items():
    table = []
    emdwtable = []
    col = []
    for entry,condprob in childmledict.items():
        row = []
        emdwrow = []
        truecol = []
        for i,val in enumerate(entry[1]):
            row.append(val)
            truecol.append(entry[0][i])
            emdwrow.append(emdwrvcategorydict[entry[0][i]][val])
        row.append(condprob)
        emdwrow.append(condprob)
        table.append(row)
        emdwtable.append(emdwrow)
        col = truecol
    familytable[child] = table
    familycolumns[child] = col
    emdwfamilytable[child] = emdwtable
    
    
for child, table in familytable.items():
    df = pd.DataFrame(data=table, columns=familycolumns[child] + ['condprob'])
    name = child + "_given_"
    for parent in family[child]:
        name = name +"_" + parent
    name = name + ".csv"
    print(name)
    df.to_csv(name,sep=',')
    
dfdict = {}
for child, emdwtable in emdwfamilytable.items():
    df = pd.DataFrame(data=emdwtable, columns=familycolumns[child] + ['condprob'])
#     df = pd.DataFrame(data=emdwtable)
    name = "emdw_" + child + "_given_"
    for parent in family[child]:
        name = name +"_" + parent
    name = name + ".csv"
    print(name)
    dfdict[child] = df
    df.to_csv(name,sep=',',index=False,header=False)
    
    


buying_given__maint_lug_boot.csv
verdict_given__buying_doors_persons_safety.csv
emdw_buying_given__maint_lug_boot.csv
emdw_verdict_given__buying_doors_persons_safety.csv


In [18]:
for child in dfdict:
    print(dfdict[child].to_string(index=False))

buying  maint  lug_boot  condprob
     1      0         0  0.200000
     0      1         2  0.400000
     3      1         2  0.400000
     0      3         0  0.250000
     0      2         2  0.265306
     3      2         2  0.265306
     0      0         2  0.216667
     2      1         1  0.400000
     0      0         0  0.200000
     3      0         1  0.208791
     1      2         0  0.280000
     0      3         1  0.250000
     2      1         2  0.400000
     1      1         0  0.400000
     3      3         0  0.250000
     3      0         2  0.216667
     0      3         2  0.250000
     1      2         2  0.265306
     2      2         1  0.265306
     0      2         0  0.280000
     1      1         2  0.400000
     3      1         0  0.400000
     1      3         0  0.250000
     2      3         1  0.250000
     1      2         1  0.265306
     3      0         0  0.200000
     3      1         1  0.400000
     1      1         1  0.400000
     3      3 